In [3]:
import io #for input and output operations
import requests #to get() the PDFs or url

import pdfplumber #package for reading info from the PDFs
import re #regular expressions allows a user to search for strings
import pandas as pd
from string import digits

In [4]:
url = 'https://www.whitehouse.gov/wp-content/uploads/2023/04/Justice40-Covered-Programs-List_v1.4_04-20-2023.pdf' #website containing PDFs
r = requests.get(url) #retrieve url
f = io.BytesIO(r.content) #manipulate binary data in-memory; this functions the same as with open(file, 'rb') but serves as a temporary buffer

In [5]:
def extract_text_from_pdf(pdf):
    extracted_text = ""
    for page in pdf.pages:
        extracted_text += page.extract_text()
    return extracted_text

with pdfplumber.open(f) as pdf:
# Extract text from the entire PDF
    text = extract_text_from_pdf(pdf)

In [6]:

bureau_keywords = ["Army Corps", "Covered programs list for AmeriCorps", "Agricultural Research Service (ARS)","Farm Service Agency (FSA)","Forest Service","National Institute of Food and Agriculture (NIFA)","Natural Resources Conservation Service (NRCS)","Risk Management Agency (RMA)","Rural Business-Cooperative Service (RBCS)","Rural Housing Service (RHS)","Rural Utilities Service (RUS)","Departmentwide","National Oceanic and Atmospheric Administration (NOAA)",
"Economic Development Administration (EDA)","National Institute of Standards and Technology (NIST)","Advanced Research Projects Agency - Energy (ARPA-E)","Bonneville Power Administration (BPA)","Federal Energy Management Program (FEMP)","Grid Deployment Office (GDO)","Loan Programs Office (LPO)","Office of Clean Energy Demonstrations (OCED)","Office of Cybersecurity, Energy Security, and Emergency Response (CESER)","Office of Economic Impact and Diversity (ED)","Office of Electricity (OE)","Office of Energy Efficiency and Renewable Energy (EERE)",
"Office of Environmental Management (EM)","Office of Fossil Energy and Carbon Management (FECM)","Office of Indian Energy Policy and Programs (IE)","Office of Legacy Management (LM)","Office of Manufacturing and Energy Supply Chains (MSC)","Office of Nuclear Energy (NE)","Office of Safety, Infrastructure, and Operations (NNSA)","Office of Science (SC)","Office of State and Community Energy Programs (SCP)", "Office of Technology Transitions (OTT)","Southeastern Power Administration (SEPA)","Southwestern Power Administration (SWPA)",
"Western Area Power Administration (WAPA)","Centers for Disease Control and Prevention","National Institutes of Health","Administration for Children and Families","Assistant Secretary for Administration","Federal Emergency Management Agency","Covered programs list for HUD","Bureau of Land Management","Bureau of Reclamation", "Bureau of Indian Affairs", "Bureau of Indian Education", "Bureau of Ocean Energy Management", "Bureau of Safety and Environmental Enforcement", "Fish and Wildlife Service", "National Park Service", 
"Office of Surface Mining Reclamation and Enforcement", "Office of Insular Affairs", "Department-Wide Programs", "Employment and Training Administration", "International Commissions", "Federal Highway Administration (FHWA)", "Federal Railroad Administration (FRA)", "Federal Transportation Administration (FTA)","Maritime Administration (MARAD)","Office of the Secretary of Transportation (OST)","Veterans Benefits Administration","Covered programs list for EPA","Covered programs list for NASA", "Covered programs list for NSF",
"Appalachian Regional Commission","Delta Regional Authority","Denali Commission"]

agency_keywords = ["U.S. Army Corps of Engineers (Army Corps)","Corporation for National and Community Service (AmeriCorps)","Department of Agriculture","Department of Commerce","Department of Energy","Department of Health and Human Services","Department of Homeland Security","Department of Housing and Urban Development","Department of the Interior","Department of Labor","Department of State","Department of Transportation","Department of Veterans Affairs","Environmental Protection Agency",
"National Aeronautics and Space Administration","National Science Foundation","Appalachian Regional Commission","Delta Regional Authority","Denali Commission"]


In [7]:
def extract_numbered_lists_between_keywords(text, start_keyword, end_keyword):
    extracted_lists = []
    is_extracting = False
    current_list = []
    

    for line in text.split("\n"):
        # Check if the current line contains the start keyword
        if start_keyword.lower() in line.lower():
            is_extracting = True
            continue

        # Check if the current line contains the end keyword
        if end_keyword.lower() in line.lower():
            if current_list:
                extracted_lists.append(current_list)
                current_list = []
            is_extracting = False
            continue

        # If we are currently extracting, check if the line starts with a number
        if is_extracting and re.match(r'\d+\.\s+(.*)', line.strip()):
            agency_program[start_keyword] = line.strip()
            current_list.append(line.strip())
            
            #value = line.strip()
            
    return extracted_lists

keywords = agency_keywords
df = pd.DataFrame(columns=['agency', 'program'])

agency_program ={}

for index, key in enumerate(keywords): 

    if index < (len(keywords) - 1): #condition to not go passed the bounds of the array
        start_keyword = keywords[index]
        end_keyword = keywords[index+1]
        extracted=extract_numbered_lists_between_keywords(text, start_keyword, end_keyword)

        extracted_list = [j for i in extracted for j in i] #unpaacking list
        #print(mylist)
        for x in extracted_list:
            df=df.append({'agency':start_keyword,'program':x},ignore_index=True)
    
      
    else:
        for i in range(-3,0): #hardcoding the last 3 small agencies since the formatting is a bit wonky
            pattern = r"(?i)\b" + re.escape(keywords[i]) + r"\b(?:.*\n)*?((?:\d+\..*(?:\n|$))+)"
            matches = re.findall(pattern, text)
            extracted_list = [item.strip() for sublist in matches for item in sublist.split("\n") if item.strip()]
            for x in extracted_list:
                df=df.append({'agency':keywords[i],'program':x},ignore_index=True)



    

C:\Users\unosia\AppData\Local\Temp\ipykernel_19472\4021849253.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append({'agency':start_keyword,'program':x},ignore_index=True)
C:\Users\unosia\AppData\Local\Temp\ipykernel_19472\4021849253.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append({'agency':start_keyword,'program':x},ignore_index=True)
C:\Users\unosia\AppData\Local\Temp\ipykernel_19472\4021849253.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append({'agency':start_keyword,'program':x},ignore_index=True)
C:\Users\unosia\AppData\Local\Temp\ipykernel_19472\4021849253.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pa

In [8]:
result_df = df.drop_duplicates(subset=['program'], keep='first').reset_index(drop=True)
result_df

,agency,program
0,U.S. Army Corps of Engineers (Army Corps),1. Aquatic Ecosystem Restoration – Construction*
1,U.S. Army Corps of Engineers (Army Corps),2. Aquatic Ecosystem Restoration – Investigati...
2,U.S. Army Corps of Engineers (Army Corps),3. Flood and Storm Damage Reduction Program – ...
3,U.S. Army Corps of Engineers (Army Corps),4. Flood and Storm Damage Reduction Program – ...
4,U.S. Army Corps of Engineers (Army Corps),5. Continuing Authorities Program*
...,...,...
463,National Science Foundation,2. Coastlines and People (COPE) program
464,National Science Foundation,3. Smart & Connected Communities (S&CC)
465,Appalachian Regional Commission,1. Appalachian Regional Commission Grants*
466,Delta Regional Authority,1. Delta Regional Authority Grants*


In [9]:
result_df['program'] = result_df['program'].str.lstrip(digits)
result_df['program'] = result_df['program'].str.strip('.*""')
result_df['program'] = result_df['program'].str.replace(r'"', '')
#result_df.to_csv('J40covered.csv',index=False)

#### To Do ####
- add bureaus to list